In [2]:
import pandas as pd

df = pd.read_json("./batches_out/500_3.jsonl", lines=True, dtype={"custom_id": str})

In [ ]:

if "completion" in df:
    responses = df["completion"]
    pdf_qid = df["custom_id"].apply(lambda x: {"pdf": x.split("_")[0], "q_id": x.split("_")[1]})
else:
    df1 = pd.json_normalize(df["response"].to_numpy(), max_level=1)
    responses = df1["body.choices"].apply(lambda x: x[0]["message"]["content"] if isinstance(x, list) and len(x) > 0 else None)
    pdf_qid = df.custom_id.apply(lambda x: {"pdf": x.split("_")[0], "q_id": x.split("_")[1]})

results = []
for i in range(len(pdf_qid)):
    pdf_id = pdf_qid[i]["pdf"]
    q_id = pdf_qid[i]["q_id"]
    response = responses[i]
    
    parsed_response = {}
    current_section = None
    for item in response.split("\n"):
        item = item.strip()
        if "ANSWER" in item:
            current_section = "answer"
            if "YES" in item:
                parsed_response["answer"] = "YES"
            elif "UNSURE" in item:
                parsed_response["answer"] = "UNSURE"
            elif "NO" in item:
                parsed_response["answer"] = "NO"
            else:
                parsed_response["answer"] = item.replace("ANSWER:", "").strip()
        elif "REASONING" in item:
            current_section = "reasoning"
            parsed_response["reasoning"] = item.replace("REASONING:", "").strip()
        elif "EVIDENCE" in item:
            current_section = "evidence"
            parsed_response["evidence"] = item.replace("EVIDENCE:", "").strip()
        elif current_section:
            # Append to the current section if the item is a continuation of the previous line
            parsed_response[current_section] = (
                parsed_response.get(current_section, "") + " " + item
            )
    results.append(
        {
            "paper_id": pdf_id,
            "question_id": q_id,
            **parsed_response,
        }
    )

In [12]:
df = pd.DataFrame(results)
df.to_csv("./outputs/500_3.csv", index=False)